# Geração de Máscaras

Dentro da base LIDC-IDRI, cada exame vem acompanhado de um arquivo xml que contém um conjunto de coordenadas de marcações feitas através da avaliação de especialistas médicos, além de informações relativas ao paciente e ao diagnóstico. 

Essas marcações são utilizadas para gerar as máscaras (verdades terrestres) que são utilizadas para a validação dos métodos de segmentação. 

## Importação de Bibliotecas

In [ ]:
import csv
import untangle
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage.morphology import binary_fill_holes

In [ ]:
data_path = "/content/drive/My Drive/base/*"
output_path = "/content/drive/My Drive/mascaras-base/"
patient = glob(data_path)
patient = sorted(patient)

In [ ]:
def calc_area(mask):
    return len(np.where(mask == True)[0])

In [ ]:
for p in patient:
    subdirs = glob(p + "/*")
    # 1st Branch
    first_subdir = glob(subdirs[0]+"/*")[0]
    first_len = len([name for name in os.listdir(first_subdir + "/")])
    # 2nd Branch
    second_subdir = glob(subdirs[-1] + "/*")[0]
    second_len = len([name for name in os.listdir(second_subdir + "/")])

    # If 1st longer then build structure from 1st data
    # If 2nd longer then build structure from 2nd data
    if first_len > second_len:
        patient_xml = first_subdir + "/*.xml"
        patient_dcm = first_subdir + "/*.dcm"
    else:
        patient_xml = second_subdir + "/*.xml"
        patient_dcm = second_subdir + "/*.dcm"

    id = patient_xml.split("/")[5]
    
    markers = glob(patient_xml)
    xml = untangle.parse(markers[0])
    coords = []
    masks = []
    list_z = []
    same = []
    masks_final = []
    sub = []
    

    print("Gerando mascaras do paciente "+id)
    # Acessando os subníveis do xml para adquirir os pontos (x, y, z)
    for rs in xml.children[0].readingSession:     
        for urn in rs.unblindedReadNodule:
           
              for Roi in urn.roi:
                  sub_z = []
                  fundo = []
                  fundo = np.zeros([512, 512], dtype='int')
                  z = float(str(Roi.imageZposition.cdata))
                  sub_z.append(z)
                  sub.append(z)
                  for coord in Roi.edgeMap: 
                      fundo[int(coord.yCoord.cdata), int(coord.xCoord.cdata)] = 1 
                  if np.histogram(fundo)[0][-1] != 1:
                      if z not in list_z:
                        list_z.append(z)
                      sub_z.append(z)
                      sub.append(binary_fill_holes(fundo))
                      masks.append(sub)

    # Separa as diferentes máscaras para um mesmo nódulo
    for i in list_z:
        aux = []
        aux.append(i)
        for m in masks:
            if m[0] == i:
                aux.append(m[1])
        same.append(aux)

    # Seleciona a máscara de maior área
    for v in same:
        maior = calc_area(v[1])
        zp = v[0]
        maior_mask = v[1]
        for mp in range(2,len(v)):
            area = calc_area(v[mp])
            if area > maior:
                maior = area
                maior_mask = v[mp]
        masks_final.append([zp, maior_mask])

    # Salvando as máscaras
    print("Salvando mascaras do paciente "+id)
    os.mkdir(output_path+id)
    caminho = output_path+id+"/"
    for mask in masks_final:
        np.save(caminho +"_%s_.npy" % (str(mask[0])), mask[1])